In [2]:
import pandas as pd 

In [3]:
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')
df.head()

,Unique Key,Created Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,City,Resolution Description,Borough,Open Data Channel Type
0,20184537,2011-04-06 00:00:00,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10002.0,NEW YORK,More than one complaint was received for this ...,MANHATTAN,UNKNOWN
1,20184538,2011-04-06 00:00:00,HPD,Department of Housing Preservation and Develop...,GENERAL CONSTRUCTION,WINDOWS,RESIDENTIAL BUILDING,11236.0,BROOKLYN,The Department of Housing Preservation and Dev...,BROOKLYN,UNKNOWN
2,20184539,2011-04-06 00:00:00,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,WALLS,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN
3,54732265,2022-07-08 11:14:43,DSNY,Department of Sanitation,Dirty Condition,Trash,Sidewalk,10467.0,BRONX,The Department of Sanitation investigated this...,BRONX,PHONE
4,20184540,2011-04-06 00:00:00,HPD,Department of Housing Preservation and Develop...,NONCONST,VERMIN,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN


In [ ]:
#q1: What is the average number of daily complaints received in 2022?
#(hint: use daily resampling on a column, for example, on the 'Unique Key' column as that has no missing values)

df['Created Date'] = pd.to_datetime(df['Created Date'])

df_2022 = df[df['Created Date'].dt.year == 2022]

daily_complaints_2022 = df_2022.resample('D', on = 'Created Date').count()

average_daily_complaints_2022 = daily_complaints_2022['Unique Key'].mean()
average_daily_complaints_2022

In [5]:
#q3: On which single date were the maximum number of calls received?

daily_calls = df.groupby('Created Date').size()

max_calls_date = daily_calls.idxmax()
max_calls_date

Timestamp('2013-01-24 00:00:00')

In [6]:
#q3: On the date the maximum number of calls were received, what was the most important complaint type?

df_max_calls = df[df['Created Date'] == max_calls_date]

top_complain = df_max_calls['Complaint Type'].value_counts().idxmax()
top_complain

'HEATING'

In [7]:
#q4: Quietest month: Group the data by months, and identify the month that historically has the fewest number of calls.

monthly_calls = df.groupby(df['Created Date'].dt.month).size()

quietest_month = monthly_calls.idxmin()
quietest_month_name = pd.to_datetime(str(quietest_month),format = '%m').strftime('%b')
quietest_month_name

'Dec'

In [ ]:
#q5:Resample your time series to a daily frequency.  Perform ETS decomposition based on an additive model.  What is the value of the seasonal component on 2020-12-25 (rounded to the nearest integer)?

import statsmodels.api as sm 
df = df.set_index(pd.DatetimeIndex(df['Created Date']))
del df['Created Date']

daily_data = df['Unique Key'].resample('D').count()
decomposition = sm.tsa.seasonal_decompose(daily_data, model ='additive')

seasonal = decomposition.seasonal
seasonal_value = seasonal['2020-12-25']

rounded_seasonal_value = round(seasonal_value)
rounded_seasonal_value

In [ ]:
#q6: Calculate the autocorrelation of the number of daily calls with the number of calls the day prior, ie lag of 1.  (Use the daily series).
autocorrelation_lag_1 = daily_data.autocorr(lag=1)
autocorrelation_lag_1

In [ ]:
#q7: Forecast the daily series with a test set of 90 days using the Prophet library.  What is your RMSE on your test set?

from prophet import Prophet
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')
df_prophet = df.reset_index().rename(columns={'Created Date': 'ds', 'Unique Key': 'y'})

# Split the data into training and test sets
train = df_prophet.iloc[:-90]
test = df_prophet.iloc[-90:]

# Create and fit the Prophet model
model = Prophet()
model.fit(train)

# Create future DataFrame for 90 days
future = model.make_future_dataframe(periods=90)

# Make predictions
forecast = model.predict(future)

# Calculate RMSE on the test set
predictions = forecast['yhat'].iloc[-90:]
rmse = np.sqrt(mean_squared_error(test['y'], predictions))
print(rmse)

/opt/conda/envs/mggy8413/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/envs/mggy8413/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/envs/mggy8413/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/envs/mggy8413/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dat